In [ ]:
import raytrace
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
o = raytrace.raytrace()

In [ ]:
image = o.main(500, 500)
print(image.shape)
print(np.unique(image))
plt.imshow(image)